# Assembly: conditionals
_COSC 208, Introduction to Computer Systems, 2023-10-23_

## Announcements
* Midterm Exam 2: Mon, Oct 30 6:30pm-8:30pm

## Outline
* Warm-up
* Mapping C conditionals to assembly
* Transforming C code to goto-form
* Practice with conditionals
* Mapping C loops to assembly & transforming to goto form

## Warm-up (Q3 & Q4 from last class)

* _The following C code was compiled into assembly (using `gcc`)._

In [ ]:
/*1*/  int divide(unsigned int numerator, unsigned int denominator) {
/*2*/      int result = -1;
/*3*/      result = numerator / denominator;
/*4*/      return result;
/*5*/  }

* Q1: _For each line of assembly, indicate which original line of C code the assembly instruction was derived from._

```
000000000000076c <divide>:                      // Line
    76c:    d10083ff     sub    sp, sp, #0x20   // 1
    770:    b9000fe0     str    w0, [sp, #12]   // 1
    774:    b9000be1     str    w1, [sp, #8]    // 1
    778:    12800000     mov    w0, #0xffffffff // 2
    77c:    b9001fe0     str    w0, [sp, #28]   // 2
    780:    b9400fe1     ldr    w1, [sp, #12]   // 3
    784:    b9400be0     ldr    w0, [sp, #8]    // 3
    788:    1ac00820     udiv   w0, w1, w0      // 3
    78c:    b9001fe0     str    w0, [sp, #28]   // 3
    790:    b9401fe0     ldr    w0, [sp, #28]   // 4
    794:    910083ff     add    sp, sp, #0x20   // 4
    798:    d65f03c0     ret                    // 4
```

```
000000000000076c <divide>:
    76c:    d10083ff     sub    sp, sp, #0x20
    770:    b9000fe0     str    w0, [sp, #12]
    774:    b9000be1     str    w1, [sp, #8]
    778:    12800000     mov    w0, #0xffffffff
    77c:    b9001fe0     str    w0, [sp, #28]
    780:    b9400fe1     ldr    w1, [sp, #12]
    784:    b9400be0     ldr    w0, [sp, #8]
    788:    1ac00820     udiv   w0, w1, w0
    78c:    b9001fe0     str    w0, [sp, #28]
    790:    b9401fe0     ldr    w0, [sp, #28]
    794:    910083ff     add    sp, sp, #0x20
    798:    d65f03c0     ret
```

* Q2: _Assume the registers initially hold the following values:_
```
sp = 0xFE0
w/x0 = 100
w/x1 = 5
```
_Draw the contents of the stack and registers just prior to the execution of the last instruction (`ret`) above._

```
        0  1  2  3  4  5  6  7  8          
        +--+--+--+--+--+--+--+--+
 0xFC0  |                       |
        +--+--+--+--+--+--+--+--+
 0xFC8  |     5     |    100    |
        +--+--+--+--+--+--+--+--+
 0xFD0  |                       |
        +--+--+--+--+--+--+--+--+
 0xFD8  |           |     20    |
        +--+--+--+--+--+--+--+--+
sp = 0xFE0
w/x0 = 20
w/x1 = 100
```

<p style="height:12em;"></p>

🛑 **STOP here** after completing the above question; if you have extra time please **skip ahead** to the extra practice.

* _Why is `#0xffffffff` being stored in `w0`?_ — this is the two's complement representation of -1
* _When might this function cause an error?_ — when denominator is 0
* _How would you modify the C code to avoid an error?_ – add an if statement to verify that `denominator != 0`

## Mapping C conditionals to assembly

In [ ]:
/*1*/  int divide_safe(unsigned int numerator, unsigned int denominator) {
/*2*/      int result = -1;
/*3*/      if (denominator != 0) {
/*4*/          result = numerator / denominator;
/*5*/      }
/*6*/      return result;
/*7*/  }

```
000000000000076c <divide_safe>:                     // Line
    76c:    d10083ff     sub    sp, sp, #0x20       // 1
    770:    b9000fe0     str    w0, [sp, #12]       // 1
    774:    b9000be1     str    w1, [sp, #8]        // 1
    778:    12800000     mov    w0, #0xffffffff     // 2
    77c:    b9001fe0     str    w0, [sp, #28]       // 2
    780:    b9400be0     ldr    w0, [sp, #8]        // 3
    784:    7100001f     cmp    w0, #0x0            // 3
    788:    540000a0     b.eq   79c <divide_safe+0x30> // 3
    78c:    b9400fe1     ldr    w1, [sp, #12]       // 4
    790:    b9400be0     ldr    w0, [sp, #8]        // 4
    794:    1ac00c20     udiv   w0, w1, w0          // 4
    798:    b9001fe0     str    w0, [sp, #28]       // 4
    79c:    b9401fe0     ldr    w0, [sp, #28]       // 6
    7a0:    910083ff     add    sp, sp, #0x20       // 6
    7a4:    d65f03c0     ret                        // 6
```

```
000000000000076c <divide_safe>:
    76c:    d10083ff     sub    sp, sp, #0x20
    770:    b9000fe0     str    w0, [sp, #12]
    774:    b9000be1     str    w1, [sp, #8]
    778:    12800000     mov    w0, #0xffffffff
    77c:    b9001fe0     str    w0, [sp, #28]
    780:    b9400be0     ldr    w0, [sp, #8]
    784:    7100001f     cmp    w0, #0x0
    788:    540000a0     b.eq   79c <divide_safe+0x30>
    78c:    b9400fe1     ldr    w1, [sp, #12]
    790:    b9400be0     ldr    w0, [sp, #8]
    794:    1ac00c20     sdiv   w0, w1, w0
    798:    b9001fe0     str    w0, [sp, #28]
    79c:    b9401fe0     ldr    w0, [sp, #28]
    7a0:    910083ff     add    sp, sp, #0x20
    7a4:    d65f03c0     ret
```

* _What does the `cmp` instruction do?_ — compares a register's value to another value
* _What does the `b.eq` instruction do?_ — "jumps" (i.e., branches) to a different instruction when the compared values are equal
* _Why does the assembly check if `w0 == 0` when the C code contains `!= 0`?_ — the C code checks for the condition that must be true to execute the if body, whereas the assembly code checks for the condition that must be true to **skip over** the if body

## Transforming C code to goto-form

* _How would we express this in C code?_ — using an if statement and a `goto` statement

In [ ]:
/*1*/  int divide_safe_goto(int numerator, int denominator) {
/*2*/      int result = -1;
/*3*/      if (denominator == 0)
/*4*/          goto after;
/*5*/      result = numerator / denominator;
/*6*/  after:
/*7*/      return result;
/*8*/  }

* _What happens if the code includes an else statement?_ — if condition is true, execute the if body and skip over the else body; if condition is false, skip over the if body and execute the else body

<div style="page-break-after:always;"></div>

## Practice with conditionals

_The C code below was compiled into assembly (using `gcc`)._

In [ ]:
/* 1*/  int flip(int bit) {
/* 2*/     int result = -1;
/* 3*/     if (bit == 0) {
/* 4*/          result = 1; 
/* 5*/      } 
/* 6*/      else {
/* 7*/          result = 0;
/* 8*/      }
/* 9*/      return result;
/*10*/ }

* Q3: _For each line of assembly, indicate which original line of C code the assembly instruction was derived from._

```
000000000000071c <flip>:                            // Line
    71c:    d10083ff     sub    sp, sp, #0x20       // 1
    720:    b9000fe0     str    w0, [sp, #12]       // 1
    724:    12800000     mov    w0, #0xffffffff     // 2
    728:    b9001fe0     str    w0, [sp, #28]       // 2
    72c:    b9400fe0     ldr    w0, [sp, #12]       // 3
    730:    7100001f     cmp    w0, #0x0            // 3
    734:    54000081     b.ne   744 <flip+0x28>     // 3
    738:    52800020     mov    w0, #0x1            // 4
    73c:    b9001fe0     str    w0, [sp, #28]       // 4
    740:    14000002     b      748 <flip+0x2c>     // 5
    744:    b9001fff     str    wzr, [sp, #28]      // 7
    748:    b9401fe0     ldr    w0, [sp, #28]       // 9
    74c:    910083ff     add    sp, sp, #0x20       // 9
    750:    d65f03c0     ret                        // 9
```

```
000000000000071c <flip>:                            // Line
    71c:    d10083ff     sub    sp, sp, #0x20       // 
    720:    b9000fe0     str    w0, [sp, #12]       // 
    724:    12800000     mov    w0, #0xffffffff     // 
    728:    b9001fe0     str    w0, [sp, #28]       // 
    72c:    b9400fe0     ldr    w0, [sp, #12]       // 
    730:    7100001f     cmp    w0, #0x0            //
    734:    54000081     b.ne   744 <flip+0x28>     // 
    738:    52800020     mov    w0, #0x1            // 
    73c:    b9001fe0     str    w0, [sp, #28]       // 
    740:    14000002     b      748 <flip+0x2c>     // 
    744:    b9001fff     str    wzr, [sp, #28]      // 
    748:    b9401fe0     ldr    w0, [sp, #28]       // 
    74c:    910083ff     add    sp, sp, #0x20       // 
    750:    d65f03c0     ret                        // 
```

Q4: _Write a function called `flip_goto` that behaves the same as `flip` but matches the structure of the assembly code that will be generated for `flip`. (Hint: you'll need two `goto` statements.)_

In [ ]:
int flip_goto(int bit) {
    int result = -1;
    if (bit != 0)
        goto else_body;
    result = 1; 
    goto after_else;
else_body:
    result = 0;
after_else:
    return result;
}

<p style="height:20em;"></p>

🛑 **STOP here** after completing the above question; if you have extra time please **skip ahead** to the extra practice.

<div style="page-break-after:always;"></div>

## Mapping C loops to assembly

* Example mapping while loops to assembly

In [11]:
/* 1*/   #include <stdio.h>
/* 2*/   int pow2(int n) {
/* 3*/       int result = 1;
/* 4*/       while (n > 0) {
/* 5*/           result = result * 2;
/* 6*/           n = n - 1;
/* 7*/       }
/* 8*/       return result;
/* 9*/   }
/*10*/   int main() {
/*11*/    printf("2^10 = %d\n", pow2(10));
/*12*/}

2^10 = 1024


```
0000000000000754 <pow2>:
 754:   d10083ff        sub     sp, sp, #0x20    // 2
 758:   b9000fe0        str     w0, [sp, #12]    // 2
 75c:   52800020        mov     w0, #0x1         // 3
 760:   b9001fe0        str     w0, [sp, #28]    // 3
 764:   14000007        b       780 <pow2+0x2c>  // 4
 768:   b9401fe0        ldr     w0, [sp, #28]    // 5
 76c:   531f7800        lsl     w0, w0, #1       // 5
 770:   b9001fe0        str     w0, [sp, #28]    // 5
 774:   b9400fe0        ldr     w0, [sp, #12]    // 6
 778:   51000400        sub     w0, w0, #0x1     // 6
 77c:   b9000fe0        str     w0, [sp, #12]    // 6
 780:   b9400fe0        ldr     w0, [sp, #12]    // 4
 784:   7100001f        cmp     w0, #0x0         // 4
 788:   54ffff0c        b.gt    768 <pow2+0x14>  // 4
 78c:   b9401fe0        ldr     w0, [sp, #28]    // 8
 790:   910083ff        add     sp, sp, #0x20    // 8
 794:   d65f03c0        ret                      // 8
```

```
0000000000000754 <pow2>:
 754:   d10083ff        sub     sp, sp, #0x20
 758:   b9000fe0        str     w0, [sp, #12]
 75c:   52800020        mov     w0, #0x1
 760:   b9001fe0        str     w0, [sp, #28]
 764:   14000007        b       780 <pow2+0x2c>
 768:   b9401fe0        ldr     w0, [sp, #28]
 76c:   531f7800        lsl     w0, w0, #1
 770:   b9001fe0        str     w0, [sp, #28]
 774:   b9400fe0        ldr     w0, [sp, #12]
 778:   51000400        sub     w0, w0, #0x1
 77c:   b9000fe0        str     w0, [sp, #12]
 780:   b9400fe0        ldr     w0, [sp, #12]
 784:   7100001f        cmp     w0, #0x0
 788:   54ffff0c        b.gt    768 <pow2+0x14>
 78c:   b9401fe0        ldr     w0, [sp, #28]
 790:   910083ff        add     sp, sp, #0x20
 794:   d65f03c0        ret
```

* Goto form

In [12]:
#include <stdio.h>
int pow2_goto(int n) {
    int result = 1;
    goto loop_check;
loop_top:
    result = result * 2;
    n = n - 1;
loop_check:
    if (n > 0)
        goto loop_top;
    return result;
}
int main() {
    printf("2^10 = %d\n", pow2_goto(10));
}

2^10 = 1024


* For loop can always be expressed using a while loop
    * Assembly code generated for a for loop matches the assembly code generated for a while loop

<div style="page-break-after:always;"></div>

## Practice with loops

* _The following C code was compiled into assembly (using `gcc`)._

In [16]:
/* 1*/   #include <stdio.h>
/* 2*/   int powi(int m, int n) {
/* 3*/       int result = 1;
/* 4*/       int i = 0;
/* 5*/       while (i < n) {
/* 6*/           result *= m;
/* 7*/           i++;
/* 8*/       }
/* 9*/       return result;
/*10*/   }
/*11*/   int main() {
/*12*/     printf("5^3 = %d\n", powi(5,3));
/*13*/   }

5^3 = 125


* Q5: _Label each line of assembly code with the line number of the line of C code from which the assembly instruction was derived._

```
0000000000000714 <powi>:
 714:   d10083ff    sub sp, sp, #0x20   // 2
 718:   b9000fe0    str w0, [sp, #12]   // 2
 71c:   b9000be1    str w1, [sp, #8]    // 2
 720:   52800020    mov w0, #0x1        // 3
 724:   b9001be0    str w0, [sp, #24]   // 3
 728:   b9001fff    str wzr, [sp, #28]  // 4
 72c:   14000008    b   74c <powi+0x38> // 5
 730:   b9401be1    ldr w1, [sp, #24]   // 6
 734:   b9400fe0    ldr w0, [sp, #12]   // 6
 738:   1b007c20    mul w0, w1, w0      // 6
 73c:   b9001be0    str w0, [sp, #24]   // 6
 740:   b9401fe0    ldr w0, [sp, #28]   // 7
 744:   11000400    add w0, w0, #0x1    // 7
 748:   b9001fe0    str w0, [sp, #28]   // 7
 74c:   b9401fe1    ldr w1, [sp, #28]   // 5
 750:   b9400be0    ldr w0, [sp, #8]    // 5
 754:   6b00003f    cmp w1, w0          // 5
 758:   54fffecb    b.lt    730 <powi+0x1c> // 5
 75c:   b9401be0    ldr w0, [sp, #24]   // 9
 760:   910083ff    add sp, sp, #0x20   // 9
 764:   d65f03c0    ret                 // 9
```

```
0000000000000714 <powi>:
 714:   d10083ff    sub sp, sp, #0x20
 718:   b9000fe0    str w0, [sp, #12]
 71c:   b9000be1    str w1, [sp, #8]
 720:   52800020    mov w0, #0x1
 724:   b9001be0    str w0, [sp, #24]
 728:   b9001fff    str wzr, [sp, #28]
 72c:   14000008    b   74c <powi+0x38>
 730:   b9401be1    ldr w1, [sp, #24]
 734:   b9400fe0    ldr w0, [sp, #12]
 738:   1b007c20    mul w0, w1, w0
 73c:   b9001be0    str w0, [sp, #24]
 740:   b9401fe0    ldr w0, [sp, #28]
 744:   11000400    add w0, w0, #0x1
 748:   b9001fe0    str w0, [sp, #28]
 74c:   b9401fe1    ldr w1, [sp, #28]
 750:   b9400be0    ldr w0, [sp, #8]
 754:   6b00003f    cmp w1, w0
 758:   54fffecb    b.lt    730 <powi+0x1c>
 75c:   b9401be0    ldr w0, [sp, #24]
 760:   910083ff    add sp, sp, #0x20
 764:   d65f03c0    ret
```

Q6: _Write a function called `powi_goto` that behaves the same as `powi` but matches the structure of the assembly code that will be generated for `powi`. (Hint: you'll need two `goto` statements.)_

In [17]:
#include <stdio.h>
int powi_goto(int m, int n) {
    int result = 1;
    int i = 0;
    goto loop_check;
loop_top:
    result *= m;
    i++;
loop_check:
    if (i < n)
        goto loop_top;
    return result;
}
int main() {
    printf("5^3 = %d\n", powi_goto(5,3));
}

5^3 = 125


<div style="page-break-after:always;"></div>

## Extra practice

Q7: _Assume the registers currently hold the following values:_
```
sp = 0xA980
w/x0 = 0
w/x1 = 1
w/x2 = 2
w/x3 = 3
w/x4 = 4
w/x5 = 5
```
_Draw the contents of the stack after the following instructions have been executed:_
```
sub sp, sp, #0x30
str w0, [sp, #16]
str x1, [sp]
str w2, [sp, #20]
str x3, [sp, #32]
str w4, [sp, #28]
str w5, [sp, #8]
```

```
        0  1  2  3  4  5  6  7  8          
 sp ->  +--+--+--+--+--+--+--+--+
0xA950  |           1           |
        +--+--+--+--+--+--+--+--+
0xA958  |     5     |           |
        +--+--+--+--+--+--+--+--+
0xA960  |     0     |     2     |
        +--+--+--+--+--+--+--+--+
0xA968  |           |     4     |
        +--+--+--+--+--+--+--+--+
0xA970  |           3           |
        +--+--+--+--+--+--+--+--+
0xA978  |                       |
        +--+--+--+--+--+--+--+--+
0xA980  |                       |
        +--+--+--+--+--+--+--+--+
```

<div style="height:10em;"></div>

* Q8: _The following C code was compiled into assembly (using `gcc`). Label each line of assembly code with the line number of the line of C code from which the assembly instruction was derived._ 

In [ ]:
/* 1*/   int abs(int value) {
/* 2*/       if (value < 0) {
/* 3*/           value = value * -1;
/* 4*/       }
/* 5*/       return value;
/* 6*/   }

```
000000000000074c <abs>:
    74c:   d10043ff        sub     sp, sp, #0x10    // 1
    750:   b9000fe0        str     w0, [sp, #12]    // 1
    754:   b9400fe0        ldr     w0, [sp, #12]    // 2
    758:   7100001f        cmp     w0, #0x0         // 2 
    75c:   5400008a        b.ge    76c <abs+0x20>   // 2
    760:   b9400fe0        ldr     w0, [sp, #12]    // 3
    764:   4b0003e0        neg     w0, w0           // 3
    768:   b9000fe0        str     w0, [sp, #12]    // 3
    76c:   b9400fe0        ldr     w0, [sp, #12]    // 5
    770:   910043ff        add     sp, sp, #0x10    // 5
    774:   d65f03c0        ret                      // 5
``` 

```
000000000000074c <abs>:
    74c:   d10043ff        sub     sp, sp, #0x10
    750:   b9000fe0        str     w0, [sp, #12]
    754:   b9400fe0        ldr     w0, [sp, #12]
    758:   7100001f        cmp     w0, #0x0 
    75c:   5400008a        b.ge    76c <abs+0x20>
    760:   b9400fe0        ldr     w0, [sp, #12]
    764:   4b0003e0        neg     w0, w0
    768:   b9000fe0        str     w0, [sp, #12]
    76c:   b9400fe0        ldr     w0, [sp, #12]
    770:   910043ff        add     sp, sp, #0x10
    774:   d65f03c0        ret
``` 

* Q9: _Write a function called `abs_goto` that behaves the same as `abs` but matches the structure of the assembly code that is generated for `abs`._

In [ ]:
int abs(int value) {
    if (value >= 0)
        goto after_if;
    value = value * -1;
after_if:
    return value;
}